#### Import Packages

In [1]:
import numpy as np
import statsmodels as sm
from scipy.stats import ttest_1samp
from statsmodels.stats.power import TTestPower
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import plotly.express as px

#### Initialize Parameters

In [2]:
#setting parameters
nSims = 100000 # Number of simluations we want to test
M = 106 # This is the mean score in the sample and would be compared with 100 in a one-sample t-test
n = 70 # Sample Size
SD = 15 # standard deviation
p_value_list = [] # Initialize a list that will store all the p-values
np.random.seed(1)

#### Simulate the experiment

We use a for loop to simulate the experiment for the number of times we want to run the experiment.

In this case, we simulate the experiment for 100K times.

In [3]:
for i in range(1,nSims):
    x = np.random.normal(loc=M, scale=SD, size=n)
    t_stat, p_value = ttest_1samp(x, popmean=100)
    p_value_list.insert(i,p_value)

In [4]:
def condition(x):
    return x < 0.01

power = sum(condition(x) for x in p_value_list)/nSims
print(f'The power calculated by summing significant p-values = {np.round(power*100,0)}%')

The power calculated by summing significant p-values = 76.0%


In [6]:
# calculate the power using statsmodels TTestPower()

power = TTestPower().power(effect_size=(M-100)/SD, nobs=n, alpha=0.01, df=None, alternative= 'two-sided')
print(f'The power calculated by the statsmodels TTtestPower = {np.round(power*100,0)}%')

The power calculated by statsmodels TTtestPower = 75.0%


In [7]:
hist_df = pd.DataFrame({"p_values":p_value_list})
bars = 20

# fig = sns.histplot(hist_df, x = "p_values")
# fig.set(ylim=(0, 100000))

fig = px.histogram(hist_df, x="p_values")
fig.update_traces(xbins=dict( # bins used for histogram
        start=0.0,
        end=1.0,
        size=0.01
    ))
fig.update_layout(yaxis_range=[0,100000], yaxis_title="Frequency of p-values")
fig.add_hline(y=nSims/bars, line_width=3, line_dash="dash", line_color="red")
fig.show()